<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/kanagawa/kanagawa_daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pandas as pd
import requests
from urllib.parse import urljoin

In [2]:
dt_now = datetime.datetime.now()

In [3]:
def str2date(s: pd.Series) -> pd.Series:

    df = s.str.extract("(\d{1,2})/(\d{1,2})").rename(columns={0: "month", 1: "day"}).fillna(0).astype(int)

    df["year"] = dt_now.year

    tmp = pd.to_datetime(df, errors="coerce")

    df["year"] = df["year"].mask(tmp > dt_now, df["year"] - 1)

    return pd.to_datetime(df, errors="coerce")

In [4]:
url = "http://www.pref.kanagawa.jp/osirase/1369/data/data.json"

In [5]:
r = requests.get(url)

In [6]:
data = r.json()

# 宿泊施設

In [7]:
link = urljoin(url, data["capacity"]["daily_capacity"]["宿泊施設CSVファイル"][0]["name"])

In [8]:
df1 = pd.read_csv(link, index_col="日付", parse_dates=True, encoding="cp932")

# 入院者数・宿泊療養者数

In [9]:
df_tmp = pd.json_normalize(data["inspection"]["daily_bed_count_by_case"]["items"]).set_index("label")

In [10]:
df2 = df_tmp["values"].apply(pd.Series).T.set_axis(data["inspection"]["daily_bed_count_by_case"]["labels"])

In [11]:
df2

label,重症者数（入院）,中等症・軽症者数（入院）,宿泊療養者数
07/01,8,41,30
07/02,7,51,36
07/03,7,50,42
07/06,10,58,51
07/07,7,52,51
...,...,...,...
01/25,106,849,285
01/26,110,825,267
01/27,105,823,250
01/28,104,828,272


In [12]:
df2.index = str2date(df2.index)

# 結合

In [13]:
df = df1.join(df2).dropna(thresh=6).astype(int)

In [14]:
df.to_csv("kanagawa.csv", encoding="utf_8_sig")